# STO twist

In [1]:
from interfacemaster.cellcalc import get_primitive_hkl, rot
from interfacemaster.interface_generator import core, convert_vector_index
from numpy import array, dot, round, var, average, pi
from numpy.linalg import inv, norm
from gb_code.csl_generator import print_list

/Users/jasonxie/.local/lib/python3.8/site-packages/scipy/__init__.py:138: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion} is required for this version of "


## Define an interface core class

Input two cif files.For GBs, input the same file; for interfaces, input two different files.


The first structure is non-changed while the second one can be deformed to form an approximate CSL

Please input files of CONVENTIONAL cells if you are not familiar with the indices in the primitive cell.

In [2]:
my_interface = core('cif_files/SrTiO3.cif','cif_files/SrTiO3.cif')

Warning!, this programme will rewrite the POSCAR file in this dir!


## Specify the rotation axis and searching parameters to find an approximate CSL

Since this is for a cubic lattice, we have determined rotation axis/angle to give a specific CSL

In [3]:
print_list([0,0,1],100)

Sigma:     1  Theta:   0.00 
Sigma:     5  Theta:  36.87 
Sigma:    13  Theta:  22.62 
Sigma:    17  Theta:  28.07 
Sigma:    25  Theta:  16.26 
Sigma:    29  Theta:  43.60 
Sigma:    37  Theta:  18.92 
Sigma:    41  Theta:  12.68 
Sigma:    53  Theta:  31.89 
Sigma:    61  Theta:  10.39 
Sigma:    65  Theta:  14.25 
Sigma:    73  Theta:  41.11 
Sigma:    85  Theta:   8.80 
Sigma:    89  Theta:  25.99 
Sigma:    97  Theta:  42.08 


In [4]:
#rotation matrix
R = rot([0,0,1],10.39/180*pi)

In [5]:
#take this as an 'exact' CSL
my_interface.search_fixed(R, exact = True, tol = 1e-3)

Congrates, we found an appx CSL!

U1 = 
[[ 0  5  6]
 [ 0 -6  5]
 [ 1  0  0]]; sigma_1 = 61

U2 = 
[[ 0  6  5]
 [ 0 -5  6]
 [ 1  0  0]]; sigma_2 = 61

D = 
[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]




'U1', 'U2' are two sets of indices of the CSL in the two crystals' frames,
'D' is the deformation applied to the crystal 2 to form an approximate CSL

As you can see, no strain is applied to any crystal for this exact CSL

## Convert the miller indices to be expressed in the primitive cell

Now we need to specify the orientation of our interface, which is by giving a miller indices in expressed in the primitive cell.

In [6]:
"""
The interface plane is the [1,0,0] plane in the conventional cell and now we \
transform it into primitive cell
"""
hkl = get_primitive_hkl(array([0,0,1]), my_interface.conv_lattice_1, my_interface.lattice_1)
hkl

array([ 0,  0, -1])

## Compute the indices of the two slabs to make bicrystal

Now let's compute the indices of the two slabs forming the interface
If you want a near orthogonal bicrystal, please set 'orthogonal to yes, 
and adjust the lim & tol.

In [7]:
my_interface.compute_bicrystal(hkl, normal_ortho = True, plane_ortho = True, lim = 50, tol_ortho = 1e-3, tol_integer = 1e-3)

hkl in CSL: [-1  0  0]
cell 1:
[[ 0  5 -6]
 [ 0 -6 -5]
 [-1  0  0]]
cell 2:
[[ 0  6 -5]
 [ 0 -5 -6]
 [-1  0  0]]


Here we obtained the two sets of indices to build a bicrystal

## Make a bicrystal!

You can adjust the bicrystal by
1. do expansion by set 'xyz' = [dimX, dimY, dimZ]
2. do RBT by 'dzdy' = a translation vector 
(please confine dzdy in the interface plane, 
for RBT in the normal, please use 'dp1' and 'dp2', to make vacuum in the interface use 'dx');
3. introduce vacuum in the interface by 'dx' = the distance spacing the two crystals
4. adjust the termination by 'dp1', 'dp2' = length of shift of the position of 
the termination from the interface into the bulk
5. introduce vacuum to make a surface in the end of the bicrystal by
'vx' = length of the vaccumn

default: xyz = [1,1,1] and others = 0

In [10]:
my_interface.get_bicrystal(xyz_1 = [10,1,1], xyz_2 = [2,1,1], filetype='VASP', dp1 = 0, dp2 = 0)

Good! You have made a 'POSCAR' file of this bicrystal, we also generated two files named 'cell_1.cif' and 'cell_2.cif' providing the structure of the two slabs.

In [11]:
import shutil
try:
    shutil.rmtree('CNID_inputs')
except:
    print('No existing files')
#To sample the CNID, try
my_interface.sample_CNID(grid = [4,4], xyz_1 = [1,1,1], xyz_2 = [1,1,1])

CNID
[[-0.  1.]
 [ 1. -0.]
 [-1. -1.]]
making 16 files...
completed


Please check the generated 'POSCAR.x.y' files. 
The CNID here is expressed in the primitive cell's frame.
You can refer to the cartesian coordinates or in the conventional cell's frame

In [12]:
CNID_cartesian = round(my_interface.CNID,8)
CNID_cartesian

array([[-2.734364, -2.734364],
       [ 2.734364,  0.      ],
       [ 0.      ,  2.734364]])

In [13]:
CNID_conv = round(dot(inv(my_interface.conv_lattice_1)\
                              , my_interface.CNID),8)
CNID_conv

array([[-0.5, -0.5],
       [ 0.5,  0. ],
       [-0. ,  0.5]])